In [7]:
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.conf import SparkConf

spark = SparkSession.builder.config('spark.executor.cores','2').master("local").appName("RankTimeSeries").getOrCreate()

# 11809233 


In [11]:
from pyspark.sql.functions import unix_timestamp,rank, desc
from pyspark.sql.window import Window

year = 2015
no_chunks = 100

df = spark.read.format("csv").option("header", "true").load('data/Parking_Violations_Issued_-_Fiscal_Year_{}.csv'.format(year))

Dates = df.select('Summons Number', unix_timestamp('Issue Date', 'MM/dd/yyyy').alias('date')).fillna(0)
oldColumns = Dates.schema.names
newColumns = ["SN", "date"]
Dates = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), Dates)
#Dates.printSchema()
#Dates.show()
Dates.createOrReplaceTempView("initial")
Ranked = spark.sql("""
    SELECT i.SN, ROW_NUMBER() OVER (ORDER BY i.date DESC) AS position, FLOOR(RAND()*{number_of_chunks}) as chunk
    FROM initial as i
    """.format(number_of_chunks = no_chunks))
Ranked.createOrReplaceTempView("initial")


Chunk0 = spark.sql("""
    SELECT i.SN, i.position
    FROM initial as i
    WHERE i.chunk = 0
""").persist()
Chunk1 = spark.sql("""
    SELECT i.SN, i.position
    FROM initial as i
    WHERE i.chunk = 1
""")

#Ranked = Dates.withColumn("position", rank().over(window))
#Ranked.coalesce(1)

#Ranked.write.format('com.databricks.spark.csv').save('data/ranked',mode='overwrite')



In [3]:
RDD.persist

In [4]:

X = Chunk0.rdd.collect() 

In [13]:
Chunk0.coalesce(1).write.csv('C:\\Users\\Administrator\\DEnv\\MLKeras\\data\\ranked')

Py4JJavaError: An error occurred while calling o371.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperation

In [ ]:
df = spark.read.format("csv").option("header", "true").load('data/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv')

Dates = df.select('Summons Number')
window = Window.orderBy(desc('Summons Number'))
Ranked = Dates.withColumn("position", rank().over(window))
print(Ranked.count())

